This is a demo notebook for the [bcbcpy](https://github.com/aheritianad/BootCamp-BlockChain-and-Python/bcbcpy) package.

In [1]:
import os, sys

sys.path.append(os.path.abspath(".."))

# Cryptography


In [2]:
from bcbcpy import crypto


## Hashing


### `Hash` function


In [3]:
text_to_be_hashed = "Hello World"
first_hash = crypto.hash_function(text_to_be_hashed)
first_hash


'1370eeaaba7a6c7a234b1f82cc3b6d013a0088fd5e16408300f05b28b0015463'

In [4]:
crypto.hash_function(text_to_be_hashed + "!")


'e59f8bdf1305e382a4919ccefd613d3eebae612aa4c443f3af2d65663de3b075'

In [5]:
crypto.hash_function("Hello World!")


'e59f8bdf1305e382a4919ccefd613d3eebae612aa4c443f3af2d65663de3b075'

In [6]:
long_text = """
Hello everyone!

Welcome to this tutorial.
I promise that I will do my best to help you.

It will be interactive, so please participate as much as you can.

Please do not hesitate to ask if there you have questions.

Hope you will enjoy it!

Cheers!

Heritiana.
"""


In [7]:
hash_long = crypto.hash_function(long_text)
hash_long


'0c8359860bef1b26f811bf8448c546c2241c7c695f9b8ee76c1a8bf2d5b2dff3'

In [8]:
len(hash_long) == len(first_hash)


True

### Validation


In [9]:
difficulty = 4
crypto.is_valid_hash(first_hash, difficulty)


False

In [10]:
fake_hash = "0000" + first_hash[4:]
fake_hash

'0000eeaaba7a6c7a234b1f82cc3b6d013a0088fd5e16408300f05b28b0015463'

In [11]:
crypto.is_valid_hash(fake_hash, difficulty)


True

### Nonce


In [12]:
second_hash, nonce = crypto.hash_nonce_initializer(
    text_to_be_hashed, difficulty=difficulty
)
second_hash

'0000bfe6af4232f78b0c8eba37a6ba6c17b9b8671473b0b82305880be077edd9'

In [13]:
nonce


107105

In [14]:
crypto.is_valid_hash(second_hash, difficulty)


True

In [15]:
third_hash = crypto.hash_function(text_to_be_hashed, nonce=nonce)
third_hash


'0000bfe6af4232f78b0c8eba37a6ba6c17b9b8671473b0b82305880be077edd9'

In [16]:
second_hash == third_hash


True

In [17]:
forth_hash = crypto.hash_function(text_to_be_hashed, nonce, nonce="")  # type:ignore
second_hash == forth_hash

True

### What if such `nonce` does not exist?


**Think about it.**


## Crypto


### `Encryption` & `Decryption`


$$f\circ f^{-1} (x) = f^{-1}\circ f(x) = x,\ \forall x$$


In [18]:
encrypt = lambda x: 2 * x - 1
decrypt = lambda x: 0  # TODO to fill

In [19]:
import random
try:
    for i in range(20):
        x = random.randint(-5000, 5000)
        assert (
            encrypt(decrypt(x)) == decrypt(encrypt(x)) == x
        ), f"Oops! fail at {i+1}-th attempt for x = {x}"
except AssertionError as e:
    print(e)
else:
    print("Congrats! I passes the test")

Oops! fail at 1-th attempt for x = -495


In [20]:
clear_message = (
    "This is a clear message to encrypt and decrypt. I add this to make it longer."
)

### Types


#### Key


$\mathcal{F}=\{ f_\theta \:\ \theta \in \Theta\}$

$\left(f_{\theta}\right) ^{-1} = f_{\theta^\prime}$ for some $\theta^\prime \in \Theta$

$\theta \in \Theta$ : key


#### Symmetric


##### Cesar


In [21]:
cesar_key = crypto.CesarKey.generate_key(max_length=2)
print(cesar_key)


CesarKey(2, -2)


In [22]:
encr_cesar = cesar_key.encrypt(clear_message)

print(
    f"""
original:
========
{clear_message}

Encrypted:
========
{encr_cesar}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Encrypted:
Vjku"ku"c"engct"oguucig"vq"gpet{rv"cpf"fget{rv0"K"cff"vjku"vq"ocmg"kv"nqpigt0



In [23]:
decr = cesar_key.decrypt(encr_cesar)

print(
    f"""
original:
========
{clear_message}

Decrypted:
=========
{decr}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



##### Permutation


In [24]:
perm_key = crypto.PermutationKey.generate_key(length=3, n_runs=1)
print(perm_key)


PermutationKey((3, 2, 1), (3, 2, 1))


In [25]:
encr_perm = perm_key.encrypt(clear_message)

print(
    f"""
original:
========
{clear_message}

Encrypted:
========
{encr_perm}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Encrypted:
ihTi sa slc raeem ass eg otcnepyra t dncedpyr .ta I ddihtt sm oekati ol egnr.



In [26]:
decr = perm_key.decrypt(encr_perm)
print(
    f"""
original:
========
{clear_message}

Decrypted:
=========
{decr}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



##### Adding noise


In [27]:
from bcbcpy import utils


In [28]:
noisy_message = utils.add_noises(clear_message)
noisy_message


'T hEi#sU Ai%sV vaz Zc!l>e\ta_r4 HmzeHs\ns"a(gWek \nt,oc demnOcer]y.pQtG saen8d; ^dQe!c$rJy\np6t7.- qIg :a&dCdr DtlhTi{s\t Kt8oA EmQa|k0eT &iQt- Fldozn\'gmeXr>.N'

In [29]:
utils.remove_noises(noisy_message)


'This is a clear message to encrypt and decrypt. I add this to make it longer.'

In [30]:
encr_noisy = cesar_key.encrypt(noisy_message)
decr_noisy = cesar_key.decrypt(encr_noisy)
decr_without_noise = utils.remove_noises(decr_noisy)

print(
    f"""
\t\t--- CESAR --
Original:
========
{clear_message}

Noisy message:
=============
{noisy_message}

Noisy Encrypted:
===============
{encr_noisy}

Noisy Decrypted:
===============
{decr_noisy}


Cleaned Decrypted:
=================
{decr_without_noise}
"""
)


		--- CESAR --
Original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Noisy message:
T hEi#sU Ai%sV vaz Zc!l>e	a_r4 HmzeHs
s"a(gWek 
t,oc demnOcer]y.pQtG saen8d; ^dQe!c$rJy
p6t7.- qIg :a&dCdr DtlhTi{s	 Kt8oA EmQa|k0eT &iQt- Fldozn'gmeXr>.N

Noisy Encrypted:
V"jGk%uW"Ck'uX"xc|"\e#n@g cat6"Jo|gJu!u$c*iYgm"!v.qe"fgopQegt_{0rSvI"ucgp:f="`fSg#e&tL{!r8v90/"sKi"<c(fEft"FvnjVk}u "Mv:qC"GoSc~m2gV"(kSv/"Hnfq|p)iogZt@0P

Noisy Decrypted:
T hEi#sU Ai%sV vaz Zc!l>e	a_r4 HmzeHs
s"a(gWek 
t,oc demnOcer]y.pQtG saen8d; ^dQe!c$rJy
p6t7.- qIg :a&dCdr DtlhTi{s	 Kt8oA EmQa|k0eT &iQt- Fldozn'gmeXr>.N


Cleaned Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



In [31]:
encr_noisy = perm_key.encrypt(noisy_message)
decr_noisy = perm_key.decrypt(encr_noisy)
decr_without_noise = utils.remove_noises(decr_noisy)

print(
    f"""
\t\t--- PERMUTATION ---
Original:
========
{clear_message}

Noisy message:
=============
{noisy_message}

Noisy Encrypted:
===============
{encr_noisy}

Noisy Decrypted:
===============
{decr_noisy}


Cleaned Decrypted:
=================
{decr_without_noise}
"""
)


		--- PERMUTATION ---
Original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Noisy message:
T hEi#sU Ai%sV vaz Zc!l>e	a_r4 HmzeHs
s"a(gWek 
t,oc demnOcer]y.pQtG saen8d; ^dQe!c$rJy
p6t7.- qIg :a&dCdr DtlhTi{s	 Kt8oA EmQa|k0eT &iQt- Fldozn'gmeXr>.N

Noisy Encrypted:
h T#iE Us%iA VszavcZ >l!a	e4r_mH Hezs
s(a"eWg
 ko,td cnmeecOy]rQp. Gteasd8n^ ;eQd$c!yJr6p
.7tq - gI&a:dCdD rhlt{iT 	s8tK AoQmEk|aTe0i& -tQlF zodg'nXem.>rN

Noisy Decrypted:
T hEi#sU Ai%sV vaz Zc!l>e	a_r4 HmzeHs
s"a(gWek 
t,oc demnOcer]y.pQtG saen8d; ^dQe!c$rJy
p6t7.- qIg :a&dCdr DtlhTi{s	 Kt8oA EmQa|k0eT &iQt- Fldozn'gmeXr>.N


Cleaned Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



#### Asymmetric


##### RSA


Security: number factorization complexity


In [32]:
rsa_key = crypto.rsa_key_demo(chunk_size=5)
print(rsa_key)


RSAPairKeys((7313, 73), HIDDEN_KEY)


In [33]:
from bcbcpy.crypto import read_from_rsa_convert

In [34]:
encr_rsa = rsa_key.encrypt(clear_message)
read_from_rsa_convert(encr_rsa)


'Go,5<j2aa8"m>ZBd$y?Tb-w\nGsk,L3\n4g-\\WnzkvSGt#HL AOGz*S&E<;^iGsy4&wIOix|<\n4h53Q0}AqX0>U~Ne+x '

In [35]:
decr_rsa = rsa_key.decrypt(encr_rsa)
read_from_rsa_convert(decr_rsa)


'This is a clear message to encrypt and decrypt. I add this to make it longer.'

##### Diffie-Hellman

Security: discrete logarithm complexity


##### ECC


### Communication & Security


#### Nodes generation

In [36]:
from bcbcpy.node import Node


In [37]:
key = crypto.rsa_key_demo()
node = Node(key)
node.id

'user_656110'

In [38]:
node

In [39]:
class RepNode(Node):
    def __repr__(self) -> str:
        return f"{self.id} : {self._Node__keys}"  # type:ignore

In [40]:
akey = crypto.CesarKey.generate_key()
alice = RepNode(akey, "Alice")
alice


Alice : CesarKey(89, -89)

In [41]:
bkey =  crypto.PermutationKey.generate_key(10)
bob = RepNode(username="Bob", keys=bkey)
bob

Bob : PermutationKey((8, 3, 5, 2, 4, 9, 1, 10, 6, 7), (7, 4, 2, 5, 3, 9, 10, 1, 6, 8))

In [42]:
rkey = crypto.rsa_key_demo()
random_guy = RepNode(rkey)
random_guy

user_545938 : RSAPairKeys((7313, 73), HIDDEN_KEY)

#### Communication

In [43]:
sender = alice
receiver = bob

plain_message = (
    "Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)"
)

In [44]:
cipher = sender.encrypt(plain_message, key=receiver.pub)
cipher

'197085)n62at: (u4t3c0his  o c n$100eostd e seaml P ice.Ae li This! HsBbleolo'

In [45]:
receiver.decrypt(cipher)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

#### Attack

##### Reading attack


In [46]:
hacker = random_guy

In [47]:
crypto.rsa.read_from_rsa_convert(
    hacker.decrypt(cipher)
)

'Gb-"HS)E#\'e"W3MC`I8pChR4,3WPL*goCufk)P80r;QlE8u^wI0ux6<>GV]z~\\fV#3*.XMeEGq<.'

In [48]:
hacker.encrypt(cipher, bob.pub)

'!olleBo b AThisis Heaice.Pl  mee sed nlo $100 t0souhis cca 45t: (231t)60987n'

##### Q: what if `bob.pub == bob.priv`?

##### Sending attack


In [49]:
scam_message = (
    "Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)"
)

In [50]:
scam_cipher= random_guy.encrypt(scam_message, key=receiver.pub)
scam_cipher

'024136)n59at: (u7t8c0his  o c n$100eostd e seaml P ice.Ae li This! HsBbleolo'

In [51]:
receiver.decrypt(scam_cipher)

'Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)'

### Authentication


#### Encryption

In [52]:
signed_message = sender.sign(plain_message) # encrypt with sender.priv
cipher_signed = sender.encrypt(signed_message, key=receiver.pub)
cipher_signed

'9A?8@=1v>:i|B(0}<|;k8pq{((w(k(v,988mw{|l(m({miut(X(qkm6Im(tq(\\pq{)(P{Jjtmwtw'

In [53]:
sender.sign(plain_message) == sender.decrypt(plain_message)

True

In [54]:
with_noise = utils.add_noises(plain_message)
signed_with_noise = sender.sign(with_noise) 
cipher_signed_with_noise = sender.encrypt(signed_with_noise, key=receiver.pub)
cipher_signed_with_noise

"[A\n1s:R@>8><2'/O?;9=)(~_[O:Bv0@w_a3L|ki}E{/}}akq|('w8[r:p|8(B8NaCB(9(8Uuu5@C,(vmZ{%Tx>l({mBm9*37mt(ikmM:6JXkt6 ({t8Qq{(I2qJdF3qp({{jiz}<\\w()Itmo\thJtPwm4"

##### Shortcut

In [55]:
cipher_signed == sender.sends(plain_message, _to=receiver.pub)

True

In [56]:
cipher_signed == sender.sends(plain_message, _to=receiver)

True

In [57]:
cipher_signed_with_noise_bis = sender.sends(plain_message, _to=receiver, _with_noises=True)
cipher_signed_with_noise_bis

'NoIsYnOiSyVA\\1+.%@>8t<c;u6?;9=R(G\n0#:Bv0tw\nQ#J|ki}C{^r%Gkq|(`wTO!9p|8(I8HYIM(9(8"uyUF:,(vmt{~#vnl({mkm&""?mt(i1m` L(Xkt6_(Nw-<q{(I!qg,<,qp({~j?~Vv\\w()ltTkV$JtPwmoNoIsYnOiSy'

In [58]:
cipher_signed_with_noise_bis[10:-10] # trim NoIsYnOiSy

'VA\\1+.%@>8t<c;u6?;9=R(G\n0#:Bv0tw\nQ#J|ki}C{^r%Gkq|(`wTO!9p|8(I8HYIM(9(8"uyUF:,(vmt{~#vnl({mkm&""?mt(i1m` L(Xkt6_(Nw-<q{(I!qg,<,qp({~j?~Vv\\w()ltTkV$JtPwmo'

In [59]:
cipher_signed_with_noise == cipher_signed_with_noise_bis[10:-10]

False

##### Q: Do you know why is it `False`?

#### Decryption

In [60]:
signed_message = receiver.decrypt(cipher_signed)
receiver.encrypt(signed_message, key=sender.pub)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [61]:
signed_message_with_noise = receiver.decrypt(cipher_signed_with_noise)
message_with_noise = receiver.encrypt(signed_message_with_noise, key=sender.pub) # unsign
utils.remove_noises(message_with_noise)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [62]:
trimmed_cipher_signed_with_noise = cipher_signed_with_noise_bis[10:-10]
signed_message_with_noise = receiver.decrypt(trimmed_cipher_signed_with_noise)
message_with_noise = receiver.encrypt(signed_message_with_noise, key=sender.pub) # unsign
utils.remove_noises(message_with_noise)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

##### Shortcut

In [63]:
receiver.gets(cipher_signed, _from=sender.pub)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [64]:
utils.remove_noises(receiver.gets(cipher_signed_with_noise, _from=sender))

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [65]:
receiver.gets(cipher_signed_with_noise_bis, _from=sender)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

#### Attacks

##### Reading Attack


In [66]:
crypto.read_from_rsa_convert(hacker.decrypt(cipher_signed))

'8(=?Y()Apal,q&w<wI"Z,f=hGA~53z4L@|]jxx`q0!3uA?Y\n-6)ftl~|*ld<\'2Nn9wG}=,HXM\t;J'

In [67]:
hacker.encrypt(cipher_signed, key=sender.pub)

'197085)n62at: (u4t3c0his  o c n$100eostd e seaml P ice.Ae li This! HsBbleolo'

In [68]:
hacker.encrypt(cipher_signed, key=receiver.pub)

')wttmJw(j(I\\pq{q{(Pmiqkm6Xt((umm({ml(vtw(,988(|8{w}pq{(kki(<=|B(0:;9|1>8A@?v'

In [69]:
hacker.sends(cipher_signed, _to=receiver.pub)


't";\nJw]\\XH9,MG}|=n2\'~<N*lYdlf)?6t\\nx-A3!j0ux`~q]@LA4|53IzG=fw,h\\"AZ<&q),wpa[l(?=8\n("Y'

In [70]:
hacker.gets(cipher_signed, _from=alice)


'Sx{0 57Q !9hYd$i\to4oAT{R$^5`?9v-+r,D8tUbppXi(z+m97QTf%.!^ldvt"d\\4\n*Ff1o?u5$@PETl3B{xU'

In [71]:
hacker.gets(cipher_signed, _from=bob)


't";\nJw]\\XH9,MG}|=n2\'~<N*lYdlf)?6t\\nx-A3!j0ux`~q]@LA4|53IzG=fw,h\\"AZ<&q),wpa[l(?=8\n("Y'

##### Sending Attack


In [72]:
print(f"""
Original:
========
{plain_message}

Scam:
=====
{scam_message}
"""
)


Original:
Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)

Scam:
=====
Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)



In [73]:
non_signed_scam = hacker.encrypt(scam_message, key=receiver.pub)

receiver.gets(non_signed_scam, _from=sender)

'@]ddgy:gZzyL`akyaky9da[]&yHd]Yk]yk]f\\ye]y})(((ylgyl`akyY[[gmfl2y (10/.-,+*)!'

In [74]:
non_signed_scam = hacker.encrypt(scam_message, key=sender.pub)

receiver.gets(non_signed_scam, _from=sender)

'(#!$"^&z% Sd*qyde\')QdXYcqqq_S qu!  cU_q^\\UqcU\\Q]TqcYSU\tq1U@q_DXYc8rqYqR\\_U\\2'

In [75]:
signed_scam = hacker.sends(scam_message, _to=receiver)

receiver.gets(signed_scam, _from=sender)

'Sx{:\nNTT9\' ,J"0NV0TT}^T{3=|/bU\\488XG0W 1u;W0l0o<Y/m>4\\y5JC~gD\n0$b9%}~H8r_::bmTl!rrm{xU'

##### Possible Weakness

In [76]:
_, leaked_sender_priv = akey  # key of alice = expected sender

fake_signed_scam = hacker.encrypt(scam_message, leaked_sender_priv)
cipher_fake_signed_scam = hacker.encrypt(fake_signed_scam, receiver.pub)
cipher_fake_signed_scam

'8:<9;>1v=Ai|B(0}?|@k8pq{((w(k(v,988mw{|l(m({miut(X(qkm6Im(tq(\\pq{)(P{Jjtmwtw'

In [77]:
receiver.gets(cipher_fake_signed_scam, _from=sender)

'Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)'

# Blockchain


In [78]:
from bcbcpy import blockchain

## Recap: Data Structure

### List - Array

### Linked List

## Block Data

In [79]:
from bcbcpy.blockchain import block

### Block

### Initial Block

## Chain

A submodule is available at 
```python
from bcbcpy.blockchain import chain
```

### Root of a Chain

In [80]:
chain = blockchain.RootChain()
chain.hash

'0000a456e7b5a5eb059e721fb431436883143101275c4077f83fe70298f5623d'

In [81]:
initial_data = {"INITIAL AGREEMENT": {"Bob": 1000, "Alice": 500}}

initial_data_string = utils.obj2txt(initial_data, indent=4)
chain = blockchain.RootChain(initial_data_string, difficulty=4)
chain.hash

'0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0'

### Normal Chain

### Deviated Chain

## Transaction


In [82]:
from bcbcpy.blockchain.block import transaction

### Transaction Data

In [83]:
coins = 1000
sender = RepNode(crypto.rsa_key_demo(), username="Bob")
receiver = RepNode(crypto.rsa_key_demo(), username="Alice")
transaction_message = ("""
Hello Alice, 

This is 1000 that you asked me to send last time.

Bye,
Bob
"""
)

receiver_id = receiver.id
receiver_pub = receiver.pub


prev_block = chain.last_block

In [84]:
transaction_data = transaction.TransactionData(
    coins, sender, receiver_id, receiver_pub, prev_block, transaction_message
)

print(transaction_data)

{
    "sender_id": "Bob",
    "receiver_id": "Alice",
    "amount": 1000,
    "prev_hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
    "sender_confirmation": "[\n\"5oC:#RTUXN\",\n\"=<LRVG\",\n\"!<H)\\\\t@hNjR]I_O{JlCa+SyJ\",\n\"!n}6YHVrh-d<^3R&YsY0\"\n]"
}


### Transaction Block

In [85]:
transaction_block = blockchain.TransactionBlock(transaction_data)
transaction_block.hash


'12d3cfa99e7c680d2303d0749563c9c5e8fa306ad6c843d2faa9a47c08b822bf'

#### Block Validation

In [86]:
transaction_block.mine()
transaction_block.hash

'0000359194709dd3890902d60e83260c447442fcb83999867ed2535300160952'

#### Adding to Chain

In [87]:
chain.add_block(transaction_block)

### Verification


In [88]:
print(chain)


{
    "hash": "0000359194709dd3890902d60e83260c447442fcb83999867ed2535300160952",
    "data": "{\n    \"sender_id\": \"Bob\",\n    \"receiver_id\": \"Alice\",\n    \"amount\": 1000,\n    \"prev_hash\": \"0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0\",\n    \"sender_confirmation\": \"[\\n\\\"5oC:#RTUXN\\\",\\n\\\"=<LRVG\\\",\\n\\\"!<H)\\\\\\\\t@hNjR]I_O{JlCa+SyJ\\\",\\n\\\"!n}6YHVrh-d<^3R&YsY0\\\"\\n]\"\n}"
}


## Second Transaction


In [89]:
coins = 500
sender = alice
receiver = bob
transaction_message = ("""
Hi Bob, 

Please, I am sending you back the 500 because I already had free 500 in my account.

Best,
Alice.
"""
)

In [90]:
receiver_id = receiver.id
receiver_pub = receiver.pub
prev_block = chain.last_block

transaction_data_2 = transaction.TransactionData(
    coins, sender, receiver_id, receiver_pub, prev_block, transaction_message
)

print(transaction_data_2)

{
    "sender_id": "Alice",
    "receiver_id": "Bob",
    "amount": 500,
    "prev_hash": "0000359194709dd3890902d60e83260c447442fcb83999867ed2535300160952",
    "sender_confirmation": "\"'((((*xzm~gpi{p*B(*8888;=A9A<?8All;@A8A8:l>8m@;:>8k<<?<<:nkj@;AAA@>?ml:=;=;889>8A=:*4'((((*~it}m*B(=884'((((*zmkmq~mzgx}jgsm g~it}m*B(c'((((((((@4'((((((((;4'((((((((=4'((((((((:4'((((((((<4'((((((((A4'((((((((94'((((((((984'((((((((>4'((((((((?'((((e4'((((*|ziv{ik|qwv(um{{iom*B(*dvmq6k4dvJtI|}6|{mdvivdv(quwkk(=v z(m(88((nmmtilip (zlim}i(Qm8k{p(mj(8=j|( owski(v(}u((vqlmmi{tdvmQ(4{jXiwdvdv((PJq4*'$"
}


In [91]:
transaction_block = blockchain.TransactionBlock(transaction_data_2)
transaction_block.mine() # validation
chain.add_block(transaction_block)